In [ ]:
pip install ijson

     |████████████████████████████████| 133kB 11.4MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# or val=='control'
def check(val):
  return (val=='depression' or val=='ptsd' or val=='schizophrenia' or val=='bipolar')

In [ ]:
print(check('schizophrenia'))

In [ ]:
import pandas as pd
import collections
import ijson
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
posts=[]
labels=[]
# c=0
with open("/content/drive/MyDrive/Thesis Papers/SMHD/train.jl", "rb") as f:
  for line in f:
    # c=c+1
    # if(c==20):
    #   break
    item = ijson.items(line,'posts.item.text',multiple_values=True)
    
    label = ijson.items(line,'label',multiple_values=True)
    for j in label:
      if(len(j)==2):
        for val in j:
          if(check(val)):
            labels.append(val)
            s=""
            space=" "
            for i in item:
              s=s+space+i
            posts.append(s)
      else:
        if(check(j[0])):
          # if(j[0]=="control"):
          #   c=c+1
          # if(c>12000):
          #   continue
          labels.append(j[0])
          s=""
          space=" "
          for i in item:
            s=s+space+i
          posts.append(s)

In [ ]:
df = pd.DataFrame({"Label": labels,"Posts": posts})

df.head(10)

,Label,Posts
0,bipolar,Because your workplace should be responsible ...
1,depression,I second that Yeah they were kick-ass last ye...
2,ptsd,Gollywogs.. *shivers* I'm not sure if this ap...
3,depression,I was a lesbian/bisexual (it was quite hard t...
4,ptsd,Drinking + smoking weed puts you on your ass ...
5,depression,"I see where you're coming from, and I'm judg..."
6,depression,"I'm into a bit of BDSM, not really intense bu..."
7,depression,"That ""I"" (I being every one of us including m..."
8,depression,[Do you want to know more?](https://en.wikip...
9,bipolar,"Yup, hence the whole Android article when he'..."


In [ ]:
df["Label"].value_counts()

depression       2305
bipolar          1062
ptsd              411
schizophrenia     198
Name: Label, dtype: int64

In [ ]:
# filter1=df["Label"].isin(["bipolar","depression"])
# newdf=df[filter1]

In [ ]:
x=df["Posts"]
y=df["Label"]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn import metrics

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# MultinomialNB

In [ ]:
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])
text_clf = text_clf.fit(x_train, y_train)

In [ ]:
predictions = text_clf.predict(x_test)

In [ ]:
print(metrics.confusion_matrix(y_test,predictions))

[[  0 316   0   0]
 [  0 702   0   0]
 [  0 124   0   0]
 [  0  51   0   0]]


In [ ]:
print(metrics.classification_report(y_test,predictions))

               precision    recall  f1-score   support

      bipolar       0.00      0.00      0.00       316
   depression       0.59      1.00      0.74       702
         ptsd       0.00      0.00      0.00       124
schizophrenia       0.00      0.00      0.00        51

     accuracy                           0.59      1193
    macro avg       0.15      0.25      0.19      1193
 weighted avg       0.35      0.59      0.44      1193



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(metrics.accuracy_score(y_test,predictions))

0.5884325230511316


In [ ]:
s1=[]
s1.append(df.iloc[58,1])

In [ ]:
text_clf.predict(s1)

array(['depression'], dtype='<U13')

# SVM

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
text_clf_svm = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42)),])

In [ ]:
text_clf_svm = text_clf_svm.fit(x_train, y_train)

In [ ]:
import numpy as np

In [ ]:
predicted_svm = text_clf_svm.predict(x_test)
np.mean(predicted_svm == y_test)

0.5884325230511316